In [1]:
import sys
sys.path.append('../src')

In [2]:
from covid import daily_snapshot
from pathlib import Path

import population as pop
import pandas as pd
import requests

In [3]:
pd.set_option('max_rows', 100) 

In [4]:
date = input('date in mm-dd-yyyy format')

date in mm-dd-yyyy format 04-03-2020


In [5]:
state = input('state, e.g. New Jersey')

state, e.g. New Jersey New York


### Quick glance at the data

In [21]:
df = daily_snapshot(date)
df.head()

,Unnamed: 0,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
FIPS,,,,,,,,,,,,
45001.0,NaN,Abbeville,South Carolina,US,2020-04-03 22:46:37,34.223334,-82.461707,6,0,0,0,"Abbeville, South Carolina, US"
22001.0,NaN,Acadia,Louisiana,US,2020-04-03 22:46:37,30.295065,-92.414197,72,1,0,0,"Acadia, Louisiana, US"
51001.0,NaN,Accomack,Virginia,US,2020-04-03 22:46:37,37.767072,-75.632346,8,0,0,0,"Accomack, Virginia, US"
16001.0,NaN,Ada,Idaho,US,2020-04-03 22:46:37,43.452658,-116.241552,307,3,0,0,"Ada, Idaho, US"
19001.0,NaN,Adair,Iowa,US,2020-04-03 22:46:37,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [22]:
df[df.Country_Region=='US'].describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
count,0.0,2322.000000,2322.000000,2368.000000,2368.000000,2368.000000,2368.0
mean,NaN,37.952568,-90.418414,116.379223,2.992821,4.099240,0.0
std,NaN,5.119809,14.284283,1292.875036,39.369604,199.477621,0.0
min,NaN,13.444300,-159.596679,0.000000,0.000000,0.000000,0.0
25%,NaN,34.245714,-95.931390,2.000000,0.000000,0.000000,0.0
50%,NaN,38.020890,-88.213113,7.000000,0.000000,0.000000,0.0
75%,NaN,41.496883,-82.338627,27.000000,1.000000,0.000000,0.0
max,NaN,65.508155,145.673900,57159.000000,1584.000000,9707.000000,0.0


### Aggregate by state

In [23]:
state_totals = df[df.Country_Region=='US'].groupby(by='Province_State').sum()

In [24]:
state_totals.head()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
Province_State,,,,,,,
Alabama,0.0,2139.512410,-5636.444765,1495,38,0,0
Alaska,0.0,549.064225,-1289.558627,157,3,0,0
Arizona,0.0,505.138555,-1671.948482,1937,41,0,0
Arkansas,0.0,1987.911393,-5266.351952,704,12,0,0
California,0.0,1915.653957,-6152.407712,12004,265,0,0


In [25]:
state_totals['death_rate?'] = state_totals['Deaths'] / state_totals['Confirmed'] * 100.0

In [26]:
state_totals.describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active,death_rate?
count,58.0,58.000000,58.000000,58.000000,58.000000,58.000000,58.0,57.000000
mean,0.0,1519.411428,-3619.854453,4751.482759,122.189655,167.362069,0.0,2.401692
std,0.0,1245.038349,3112.331438,14017.685794,395.557056,1274.591544,0.0,1.760712
min,0.0,0.000000,-14651.797929,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.0,511.030915,-5627.524695,336.500000,6.250000,0.000000,0.0,1.573677
50%,0.0,1287.901244,-3170.306929,944.500000,23.500000,0.000000,0.0,2.135493
75%,0.0,2515.280771,-958.649806,3405.750000,95.500000,0.000000,0.0,2.849874
max,0.0,4798.990598,145.673900,102987.000000,2935.000000,9707.000000,0.0,12.500000


In [27]:
state_covid = df[df.Province_State == state].copy()
state_covid['death_rate'] = 100 * state_covid['Deaths'] / state_covid['Confirmed']
state_columns=['Admin2', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'death_rate' ]

In [28]:
# sort_by = input('sort by one of Admin2, Confirmed, Deaths, death_rate')
sort_by = "Confirmed"

In [29]:
state_covid[state_columns].sort_values(by=sort_by, ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate
FIPS,,,,,,
36061.0,New York City,57159,1584,0,0,2.771217
36119.0,Westchester,12351,67,0,0,0.542466
36059.0,Nassau,12024,95,0,0,0.790086
36103.0,Suffolk,10154,84,0,0,0.827260
36087.0,Rockland,4289,42,0,0,0.979249
36071.0,Orange,2397,30,0,0,1.251564
36027.0,Dutchess,809,5,0,0,0.618047
36029.0,Erie,734,19,0,0,2.588556
36055.0,Monroe,464,10,0,0,2.155172


## We need to locate county data to add per 1000 capita rates.

In [30]:
data = Path('../data/counties')
state_pop_path = data / Path(state.replace(' ', '').lower() + '.csv')

In [31]:
state_pop_path

PosixPath('../data/counties/newyork.csv')

In [32]:
state_pop = pd.read_csv(state_pop_path)

In [33]:
state_with_population = pd.merge(state_covid[state_columns], state_pop,
                                 how='outer',
                                 left_on='Admin2', right_on='county')

state_with_population['fraction_confirmed'] = state_with_population['Confirmed'] / state_with_population['pop2019']   * 1000.0
state_with_population['deaths']             = state_with_population['Deaths']    / state_with_population['pop2019']   * 1000.0
state_with_population['death_rate?']        = state_with_population['Deaths']    / state_with_population['Confirmed'] * 1000.0

# sort_by = input("column to sort Confirmed, Deaths, county, pop2019, fraction_confirmed, deaths, death_rate?")
sort_by = 'fraction_confirmed'
state_with_population.sort_values(by=sort_by, ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate,Rank,county,pop2019,fraction_confirmed,deaths,death_rate?
39,Rockland,4289.0,42.0,0.0,0.0,0.979249,13.0,Rockland,323686.0,13.250496,0.129755,9.792492
56,Westchester,12351.0,67.0,0.0,0.0,0.542466,7.0,Westchester,968815.0,12.748564,0.069157,5.424662
27,Nassau,12024.0,95.0,0.0,0.0,0.790086,6.0,Nassau,1356564.0,8.863570,0.070030,7.900865
47,Suffolk,10154.0,84.0,0.0,0.0,0.827260,4.0,Suffolk,1487901.0,6.824379,0.056455,8.272602
28,New York City,57159.0,1584.0,0.0,0.0,2.771217,900.0,New York City,8443713.0,6.769415,0.187595,27.712171
33,Orange,2397.0,30.0,0.0,0.0,1.251564,12.0,Orange,378227.0,6.337464,0.079317,12.515645
12,Dutchess,809.0,5.0,0.0,0.0,0.618047,15.0,Dutchess,293894.0,2.752693,0.017013,6.180470
37,Putnam,252.0,7.0,0.0,0.0,2.777778,29.0,Putnam,99070.0,2.543656,0.070657,27.777778
48,Sullivan,168.0,5.0,0.0,0.0,2.976190,36.0,Sullivan,75211.0,2.233716,0.066480,29.761905
51,Ulster,263.0,3.0,0.0,0.0,1.140684,20.0,Ulster,179303.0,1.466791,0.016731,11.406844


In [34]:
state_with_population.sort_values(by='Deaths', ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate,Rank,county,pop2019,fraction_confirmed,deaths,death_rate?
28,New York City,57159.0,1584.0,0.0,0.0,2.771217,900.0,New York City,8443713.0,6.769415,0.187595,27.712171
52,Unassigned,0.0,955.0,0.0,0.0,inf,NaN,NaN,NaN,NaN,NaN,inf
27,Nassau,12024.0,95.0,0.0,0.0,0.790086,6.0,Nassau,1356564.0,8.863570,0.070030,7.900865
47,Suffolk,10154.0,84.0,0.0,0.0,0.827260,4.0,Suffolk,1487901.0,6.824379,0.056455,8.272602
56,Westchester,12351.0,67.0,0.0,0.0,0.542466,7.0,Westchester,968815.0,12.748564,0.069157,5.424662
39,Rockland,4289.0,42.0,0.0,0.0,0.979249,13.0,Rockland,323686.0,13.250496,0.129755,9.792492
33,Orange,2397.0,30.0,0.0,0.0,1.251564,12.0,Orange,378227.0,6.337464,0.079317,12.515645
13,Erie,734.0,19.0,0.0,0.0,2.588556,8.0,Erie,919866.0,0.797942,0.020655,25.885559
25,Monroe,464.0,10.0,0.0,0.0,2.155172,9.0,Monroe,744248.0,0.623448,0.013436,21.551724
37,Putnam,252.0,7.0,0.0,0.0,2.777778,29.0,Putnam,99070.0,2.543656,0.070657,27.777778


In [35]:
state_with_population.sort_values(by='death_rate', ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate,Rank,county,pop2019,fraction_confirmed,deaths,death_rate?
52,Unassigned,0.0,955.0,0.0,0.0,inf,NaN,NaN,NaN,NaN,NaN,inf
20,Herkimer,14.0,2.0,0.0,0.0,14.285714,40.0,Herkimer,62505.0,0.223982,0.031997,142.857143
5,Chautauqua,9.0,1.0,0.0,0.0,11.111111,23.0,Chautauqua,129656.0,0.069414,0.007713,111.111111
1,Allegany,14.0,1.0,0.0,0.0,7.142857,52.0,Allegany,47025.0,0.297714,0.021265,71.428571
2,Broome,56.0,4.0,0.0,0.0,7.142857,19.0,Broome,194402.0,0.288063,0.020576,71.428571
57,Wyoming,15.0,1.0,0.0,0.0,6.666667,55.0,Wyoming,40565.0,0.369777,0.024652,66.666667
17,Genesee,17.0,1.0,0.0,0.0,5.882353,44.0,Genesee,58112.0,0.292539,0.017208,58.823529
23,Livingston,18.0,1.0,0.0,0.0,5.555556,39.0,Livingston,63907.0,0.281659,0.015648,55.555556
36,Otsego,21.0,1.0,0.0,0.0,4.761905,43.0,Otsego,60244.0,0.348582,0.016599,47.619048
41,Schenectady,110.0,5.0,0.0,0.0,4.545455,22.0,Schenectady,154883.0,0.710214,0.032282,45.454545
